In [ ]:
from six.moves import urllib
import urllib2
import requests
from lxml import etree
import os
from zipfile import ZipFile
import re

In [ ]:
directory = 'data'

In [ ]:
def download(directory, name, id):
    url = 'https://www.opensubtitles.org/en/download/vrf-108d030f/sub/' + id
    filename = name
    filepath = os.path.join(directory, filename)
    if not os.path.exists(directory):
        print("Creating directory %s" % directory)
        os.mkdir(directory)
    if not os.path.exists(filepath):
#         print("Downloading %s to %s" % (url, filepath))
        request = requests.get(url, allow_redirects=True)
        with open(filepath, 'wb') as f:
            f.write(request.content)
        statinfo = os.stat(filepath)
        print ("Successfully downloaded", filename, statinfo.st_size, "bytes")

In [ ]:
def download_by_page(r):
    url = 'https://www.opensubtitles.org/en/search/sublanguageid-zht/offset-' + r
    html = urllib2.urlopen(url).read()
    tree = etree.HTML(html)
    rows = tree.xpath("//tr[@class='change even expandable']")
    for row in rows:
        name = row.xpath("//td[1]/strong/a/text()")
        style = row.xpath("//td[5]/span[@class='p']/text()")
        url = row.xpath("//td[5]/a/@href")[1:]
        for (n, s, l) in zip(name, style, url):
            if s == 'srt':
                name =  re.sub(r'\s+', '-', n)
                id = re.search('\d+$', u).group(0)
                download(directory, name, id)

In [ ]:
for r in xrange(0, 800000, 40):
    try:
        download_by_page(r)
    except:
        break

In [ ]:
# with ZipFile('data/21-Jump-Street-(2012)') as zfile:
#     name = [file.filename for file in zfile.infolist() if re.search('\.srt', file.filename)]
#     with zfile.open(name[0], 'rU') as file:
#         for line in file.readlines():
#             print line.decode('iso-8859-1').encode('utf8')